# Examine the effects of geography and divergence on proportion of variants shared among samples 

June 24, 2020 

We would like to determine whether samples collected in the same geographic area share more variants than expected by chance alone. In addition to the permutation test, I would also like to perform some sort of metric for whether being close together on the tree also predicts having shared variation. There are probably a few different ways to do this: 

1. Compare some sort of raw measure of sequence divergence, like hamming distance (number of differences/length of sequence)
2. Compare the branch length of the path between the 2 sequences. 
3. Compare the tmrca, where more divergent sequences will have older tmrcas.

All 3 of these could be proxies for how close together sequences are on the tree. It would be good to test this out using the Wisconsin-only build as well as the Wisconsin-focused build with other sequences in there for context.

In [1]:
import imp
import importlib, json
import glob
import re,copy,json
import Bio.Phylo
import requests
import pandas as pd 
import numpy as np

import copy
from scipy.special import binom
import datetime as dt
    
import rpy2
%load_ext rpy2.ipython

# for this to work, you will need to download the most recent version of baltic, available here 
bt = imp.load_source('baltic', '/Users/lmoncla/src/baltic/baltic/baltic.py')

In [2]:
def compute_shared_variant_proportion(sample1,sample2,df):
    shared_variants = 0
    
    s1_df = df[df['strain_name'] == sample1]
    variants_in_s1 = set(s1_df['minor_nuc_muts'].tolist())
    
    s2_df = df[df['strain_name'] == sample2]
    variants_in_s2 = set(s2_df['minor_nuc_muts'].tolist())
    
    total_variants = len(variants_in_s1) + len(variants_in_s2)
    
    if total_variants == 0:
        proportion_shared = 0
    else:
        for v in variants_in_s1:
            if v in variants_in_s2:
                shared_variants += 2

        proportion_shared = float(shared_variants/total_variants)
            
    return(proportion_shared)

## Read in VCF data and output SNVs to query into a dataframe

In [3]:
"""to load in an ipython notebook as a module, just run the following. You will now have access to all of the 
functions written in that jupyter notebook"""

%run vcf-module.ipynb

In [4]:
# read in the current date 
from datetime import date
today = date.today()
current_date = str(today.strftime("%Y-%m-%d"))

In [5]:
"""now, input the strain names file/metadata file, the directory containing the vcfs, and return the dataframess"""

strain_names_file = "/Users/lmoncla/src/ncov-WI-within-host/data/sample-metadata.tsv"
fasta_file = "/Users/lmoncla/src/ncov-WI-within-host/data/sample-avrl.fasta"
clades_file = "/Users/lmoncla/src/ncov-WI-within-host/data/clades-file-2020-08-28.txt"
vcf_directory = "/Users/lmoncla/src/ncov-WI-within-host/data/vcfs-all/"

# N transcript was a control; sample/tube 6, UW-30, has an snv at basically every single site after a certain 
# point in the genome. it has 418 SNVs!!! I'm excluding it
samples_to_ignore = ["N_transcript","6"]

# set the length of homopolymers that you want to use
homopolymer_length = 3

# set the variant percent that you want to use 
variant_percent = "0.03"

snvs_only, indels_only, all_intersection_variants,metadata_dict = return_dataframes(strain_names_file, clades_file,vcf_directory,samples_to_ignore,fasta_file, homopolymer_length)

/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [6]:
"""subset data to include only SNVs and indels > the frequency you set above"""
snvs_only = snvs_only[snvs_only['minor_frequency'] >= float(variant_percent)]
indels_only = indels_only[indels_only['minor_frequency'] >= float(variant_percent)]

In [7]:
snvs_to_query = set(snvs_only['minor_nuc_muts'])
indels_to_query = set(indels_only['minor_nuc_muts'])
all_variants_to_query = snvs_to_query.copy()
all_variants_to_query.update(indels_to_query)
print(len(snvs_to_query))
print(len(indels_to_query))
print(len(all_variants_to_query))

288
74
362


## Code for parsing through tree

In [8]:
"""This is a small, recursive function to return the TMRCA for 2 tips. Starting with the parental node of tip1,
go recursively backwards in the tree until you find an internal node whose children contains both tip1 and 
tip2. Return that node."""

def return_TMRCA_node(input_node,tip1,tip2):
    
    # for a given internal node, generate a list of all its children, i.e., tips descending from that node
    node = input_node
    children = list(node.children)   # .children will output all of the direct descendants as baltic objects
    leaves = list(node.leaves)       # .leaves will output the names of all tips descending from the node
    
    if tip2 in leaves and tip1 in leaves: 
        node_to_return = node
    else:
        node_to_return = return_TMRCA_node(node.parent,tip1,tip2)
            
    return(node_to_return)

In [9]:
"""given 2 tips and a tree, iterate through the tree. when we reach tip 1, run return_TMRCA_node, to find the 
internal node that is the TMRCA for tips 1 and 2. Extract its date and return the node object and date"""

def return_TMRCA(tip1,tip2,tree):
    for k in tree.Objects: 
        if k.branchType == "leaf" and k.name == tip1:
            tmrca_node = return_TMRCA_node(k.parent,tip1,tip2)
            date = tmrca_node.traits['node_attrs']['num_date']['value']  # output the mean inferred date
            
    return(tmrca_node, date)

In [10]:
"""Given a starting internal node, and a tip you would like to end at, traverse the full path from that node to
tip. Along the way, gather nucleotide mutations that occur along that path. Once you have reached the ending 
tip, return the list of mutations that fell along that path"""

def return_divergence_on_path_to_tip(starting_node, ending_tip):
    
    children = starting_node.children
    
    for child in children:
        
        """if the child is a leaf: if leaf is the target end tip, collect its divergence and return; 
        if leaf is not the target end tip, move on"""
        """if the child is an internal node: first, test whether that child node contains the target tips in its 
        children. child.leaves will output a list of the names of all tips descending from that node. If not, pass. 
        if the node does contain the target end tip in its leaves, keep traversing down that node recursively"""

        if child.branchType == "leaf":
            if child.name != ending_tip:
                pass
            elif child.name == ending_tip:
                child_divergence = child.traits['node_attrs']['div']
                return(child_divergence)
         
        elif child.branchType == "node":
            if ending_tip not in child.leaves:
                pass
            else:
                child_divergence = return_divergence_on_path_to_tip(child, ending_tip)
    
    return(child_divergence)

In [11]:
def return_clade(tipname, tree):
    for k in tree.Objects:
        if k.branchType == "leaf" and k.name == tipname:
            clade = k.traits['node_attrs']['clade_membership']['value']
    return(clade)

In [12]:
def return_all_Wisconsin_tips(tree):
    Wisconsin_leaves = []
    
    for k in tree.Objects: 
        if k.branchType == "leaf":
            division = k.traits['node_attrs']['division']['value']
            if division == "Wisconsin":
                Wisconsin_leaves.append(k.name)
                
    return(Wisconsin_leaves)

In [14]:
def return_mean_Ct(metadata,tip):
    Cts = []
    
    Ct1 = metadata[tip]['Ct1']
    Ct2 = metadata[tip]['Ct2']
    
    if Ct1 not in ["","-","n/a"]:
        Cts.append(Ct1)
    if Ct2 not in ["","-","n/a"]:
        Cts.append(Ct2)
    
    # now find mean 
    Ct_sum = 0
    for c in Cts: 
        Ct_sum += float(c)
        
    if len(Cts) > 0:
        mean = float(Ct_sum)/len(Cts)
    else: 
        mean = 'NaN'
    
    return(mean)

In [15]:
def compare_Cts(tip1,tip2,metadata):
    tip1_Ct = return_mean_Ct(metadata,tip1)
    tip2_Ct = return_mean_Ct(metadata,tip2)
    
    if tip1_Ct not in ['NaN','n/a'] and tip2_Ct not in ['NaN','n/a']:
        difference = abs(tip1_Ct-tip2_Ct)
    else:
        difference = 'NaN'
    return(difference)

In [16]:
def compare_location(tip1,tip2,metadata):
    geo1 = metadata[tip1]["location"]
    geo2 = metadata[tip2]["location"]
    
    if geo1 == geo2: 
        location = 1
    else: 
        location = 0
        
    return(location,geo1,geo2)

In [17]:
def return_lat_longs_dictionary(lat_longs_file):
    
    output_dict = {}
    
    with open(lat_longs_file, "r") as infile: 
        for line in infile:
            if len(line.split("\t")) == 4:
                location = line.split("\t")[1]
                longitude = line.split("\t")[2]
                latitude = line.split("\t")[3].strip()
                output_dict[location] = {"latitude":latitude, "longitude":longitude}
    return(output_dict)

In [18]:
"""I took this from here, https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
A decent overview of this formula can be found here: https://www.movable-type.co.uk/scripts/latlong.html"""
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees). 
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [19]:
def return_distance_between_locations(tip1,tip2,metadata,lat_longs):
    geo1 = metadata[tip1]["location"]
    geo2 = metadata[tip2]["location"]
    
    if geo1 != "" and geo2 != "":
        lat1 = lat_longs_dict[geo1]['latitude']
        lat2 = lat_longs_dict[geo2]['latitude']

        long1 = lat_longs_dict[geo1]['longitude']
        long2 = lat_longs_dict[geo2]['longitude']

        distance_km = haversine(float(long1), float(lat1), float(long2), float(lat2))
    else:
        distance_km = 'NaN'
    return(distance_km)

In [20]:
import itertools

def generate_all_combinations_pairs(list_of_elements):
    all_pairs = []
    for pair in itertools.combinations(list_of_elements, 2):
        all_pairs.append(list(pair))
    return(all_pairs)

In [21]:
def generate_all_permutations_of_household_groups(household_groups):
    total_pairs = []
    for household in household_groups: 
        all_pairs = generate_all_combinations_pairs(household)
        for a in all_pairs: 
            total_pairs.append(a)
    
    return(total_pairs)

In [22]:
def return_same_household(tip1,tip2,household_groups):
    total_pairs = generate_all_permutations_of_household_groups(household_groups)
    
    if [tip1,tip2] in total_pairs or [tip2,tip1] in total_pairs:
        same_household = 1
    else:
        same_household = 0
    
    return(same_household)

In [23]:
def return_probable_trans_pair(tip1,tip2,same_household,pairs_to_exclude):
    
    if same_household == 1:
        if [tip1,tip2] not in pairs_to_exclude and [tip2,tip1] not in pairs_to_exclude:
            trans_pair = 1
        else:
            trans_pair = 0
    else:
        trans_pair = 0
    
    return(trans_pair)

In [29]:
for s in set(snvs_only['strain_name'].tolist()):
    meanCt = return_mean_Ct(metadata_dict,s)
    print(s, meanCt)

USA/WI-UW-76/2020 17.54
USA/WI-UW-282/2020 15.15
USA/WI-UW-154/2020 21.5
USA/WI-UW-916/2020 NaN
USA/WI-UW-225/2020 18.73
USA/WI-UW-893/2020 NaN
USA/WI-UW-24/2020 NaN
USA/WI-UW-39/2020 19.13
USA/WI-UW-334/2020 23.015
USA/WI-UW-306/2020 30.82
USA/WI-UW-722/2020 NaN
USA/WI-UW-65/2020 15.434999999999999
USA/WI-UW-244/2020 19.45
USA/WI-UW-749/2020 NaN
USA/WI-UW-246/2020 22.064999999999998
USA/WI-UW-575/2020 NaN
USA/WI-UW-231/2020 19.68
USA/WI-UW-333/2020 26.994999999999997
USA/WI-UW-286/2020 25.145
USA/WI-UW-341/2020 22.855
USA/WI-UW-17/2020 NaN
USA/WI-UW-232/2020 18.935
USA/WI-UW-876/2020 NaN
USA/WI-UW-117/2020 28.2
USA/WI-UW-302/2020 17.34
USA/WI-UW-85/2020 24.4
USA/WI-UW-32/2020 24.549999999999997
USA/WI-UW-438/2020 19.4
USA/WI-UW-07/2020 16.384999999999998
USA/WI-UW-61/2020 23.695
USA/WI-UW-217/2020 29.37
USA/WI-UW-118/2020 31.86
USA/WI-UW-997/2020 NaN
USA/WI-UW-991/2020 NaN
USA/WI-UW-443/2020 19.4
USA/WI-UW-96/2020 17.689999999999998
USA/WI-UW-201/2020 21.21
USA/WI-UW-88/2020 25.61
USA

# run!

In [26]:
# test this out first on the Wisconsin-only build json
WI_tree_path = "/Users/lmoncla/src/ncov-WI-within-host/data/ncov_wisconsin_statewide.json"

analysis_level = "division"

with open(WI_tree_path) as json_file:
    WI_tree_json = json.load(json_file)
WI_tree_object=WI_tree_json['tree']
WI_meta=WI_tree_json['meta']
json_translation={'absoluteTime':lambda k: k.traits['node_attrs']['num_date']['value'],'name':'name'} ## allows baltic to find correct attributes in JSON, height and name are required at a minimum
#json_meta={'file':WI_meta,'traitName':analysis_level} ## if you want auspice stylings you can import the meta file used on nextstrain.org

WI_tree=bt.loadJSON(WI_tree_object,json_translation)


Tree height: 0.853113
Tree length: 155.604745
multitype tree
annotations present

Numbers of objects in tree: 5452 (2518 nodes and 2934 leaves)



The putative transmission pairs are: 
1. USA/WI-UW-65/2020 and USA/WI-UW-32/2020  (2 days apart)
2. USA/WI-UW-41/2020 and USA/WI-UW-48/2020 (same day)
3. USA/WI-UW-74/2020 and USA/WI-UW-29/2020 (4 days apart)
4. USA/WI-UW-120/2020 and USA/WI-UW-119/2020 (3 days apart)
5. USA/WI-UW-333/2020 and USA/WI-UW-334/2020 (same day)
6. USA/WI-UW-337/2020 and USA/WI-UW-338/2020 (same day) -> I am actually excluding these, because they do not have the same consensus sequence; they are 3 mutations different, which seems pretty unlikely
7. USA/WI-UW-158/2020 and USA/WI-UW-159/2020 and USA/WI-UW-160/2020  (these are all in the same household, all collected on the same day) -> I am also going to call 158 and 160 true transmission pairs because they have the same consensus sequence, while 159 has 2 additional mutations (158 and 160 isolated same day). 159 does look like it be descendant from 158 and 160, but becuase it has 2 mutations different tthey were all collected on the same day, it seems a little weird.

pairs 5 and 6 are from the Milwaukee area, whereas the rest are more from the Madison area

In [27]:
household_groups = [["USA/WI-UW-27/2020","USA/WI-UW-85/2020"],
                      ["USA/WI-UW-40/2020","USA/WI-UW-97/2020"],
                      ["USA/WI-UW-41/2020","USA/WI-UW-48/2020"],
                      ["USA/WI-UW-65/2020","USA/WI-UW-32/2020"],
                      ["USA/WI-UW-69/2020","USA/WI-UW-61/2020"],
                      ["USA/WI-UW-70/2020","USA/WI-UW-67/2020"],
                      ["USA/WI-UW-74/2020","USA/WI-UW-29/2020"],
                      #["USA/WI-UW-119/2020","USA/WI-UW-120/2020"],  # 119 is missing a consensus genome
                      ["USA/WI-UW-124/2020","USA/WI-UW-351/2020"],
                      ["USA/WI-UW-158/2020","USA/WI-UW-160/2020"],
                      ["USA/WI-UW-264/2020","USA/WI-UW-356/2020"],
                      ["USA/WI-UW-333/2020","USA/WI-UW-334/2020"],
                      ["USA/WI-UW-476/2020","USA/WI-UW-438/2020","USA/WI-UW-432/2020"],
                      ["USA/WI-UW-544/2020","USA/WI-UW-551/2020","USA/WI-UW-575/2020"],
                      ["USA/WI-UW-546/2020","USA/WI-UW-586/2020","USA/WI-UW-443/2020"],
                      ["USA/WI-UW-577/2020","USA/WI-UW-536/2020"],
                      ["USA/WI-UW-598/2020","USA/WI-UW-602/2020","USA/WI-UW-689/2020"],
                      ["USA/WI-UW-601/2020","USA/WI-UW-780/2020"],
                      ["USA/WI-UW-756/2020","USA/WI-UW-893/2020"],
                      ["USA/WI-UW-784/2020","USA/WI-UW-722/2020","USA/WI-UW-749/2020","USA/WI-UW-694/2020","USA/WI-UW-721/2020","USA/WI-UW-798/2020"],
                      ["USA/WI-UW-747/2020","USA/WI-UW-697/2020"],
                      ["USA/WI-UW-855/2020","USA/WI-UW-897/2020","USA/WI-UW-916/2020","USA/WI-UW-906/2020"],
                      ["USA/WI-UW-874/2020","USA/WI-UW-986/2020","USA/WI-UW-997/2020","USA/WI-UW-991/2020"],
                      ["USA/WI-UW-895/2020","USA/WI-UW-876/2020","USA/WI-UW-931/2020","USA/WI-UW-863/2020"],
                      ["USA/WI-UW-927/2020","USA/WI-UW-861/2020"]]


"""these are all pairs for which the 2 consensus genomes are separated by >2 mutations"""
pairs_to_exclude = [['USA/WI-UW-476/2020', 'USA/WI-UW-438/2020'], ['USA/WI-UW-476/2020', 'USA/WI-UW-432/2020'], 
                    ['USA/WI-UW-784/2020', 'USA/WI-UW-722/2020'], ['USA/WI-UW-784/2020', 'USA/WI-UW-749/2020'], 
                    ['USA/WI-UW-784/2020', 'USA/WI-UW-694/2020'], ['USA/WI-UW-784/2020', 'USA/WI-UW-721/2020'], 
                    ['USA/WI-UW-784/2020', 'USA/WI-UW-798/2020'], ['USA/WI-UW-855/2020', 'USA/WI-UW-897/2020'], 
                    ['USA/WI-UW-897/2020', 'USA/WI-UW-916/2020'], ['USA/WI-UW-897/2020', 'USA/WI-UW-906/2020']]

In [28]:
Wisconsin_tips_in_tree = return_all_Wisconsin_tips(WI_tree)
print(len(Wisconsin_tips_in_tree))

2778


In [29]:
tips_to_query = set(all_intersection_variants['strain_name'].tolist())

not_in_tree = []
for t in tips_to_query:
    if t not in Wisconsin_tips_in_tree:
        print(t)
        not_in_tree.append(t)
        
print(len(tips_to_query))

USA/DO NOT UPLOAD/2020
USA/DO NOT UPLOAD - time series sample/2020
USA/WI-UW-119/2020
142


In [30]:
for n in not_in_tree:
    tips_to_query.remove(n)
print(len(tips_to_query))

139


In [31]:
# read in metadata and latitude and longitude files
metadata_input_file = "/Users/lmoncla/src/ncov-WI-within-host/data/sample-metadata.tsv"
metadata_dict = return_metadata_dict(metadata_input_file, clades_file)
lat_longs_dict = return_lat_longs_dictionary("/Users/lmoncla/src/ncov/defaults/lat_longs.tsv")
wh_df_to_use = snvs_only
tree = WI_tree

In [36]:
df = pd.DataFrame()

combos = []
for t in tips_to_query: 
    tip1 = t
    
    for a in tips_to_query:
        tip2 = a
        combo = set([tip1,tip2])
        
        if tip1 != tip2 and combo not in combos:   # to prevent doing the pairwise comparisons twice
            # output Cts
            Ct_diff = compare_Cts(tip1,tip2,metadata_dict)
            
            # are the locations the same? 0 means no, 1 means yes
            location,loc1,loc2 = compare_location(tip1,tip2,metadata_dict)
            
            # output great circle distance between locations
            great_circle_distance = return_distance_between_locations(tip1,tip2,metadata_dict,lat_longs_dict)

            # output their clades
            tip1_clade = return_clade(tip1, tree)
            tip2_clade = return_clade(tip2, tree)
            if tip1_clade == tip2_clade:
                clades_same = 1
            else:
                clades_same = 0
                
            # are you a household pair? Are you a likely transmission pair?
            same_household = return_same_household(tip1,tip2,household_groups)
            trans_pair = return_probable_trans_pair(tip1,tip2,same_household,pairs_to_exclude)

            # output the tmrca and divergence
            parental_node,tmrca_date = return_TMRCA(tip1,tip2,tree)
            parent_divergence = parental_node.traits['node_attrs']['div']

            tip1_divergence = return_divergence_on_path_to_tip(parental_node, tip1)
            tip2_divergence = return_divergence_on_path_to_tip(parental_node, tip2)

            node_to_tip1 = tip1_divergence - parent_divergence
            node_to_tip2 = tip2_divergence - parent_divergence
            total_divergence = node_to_tip1 + node_to_tip2

            # calculate the proportion of variants shared
            shared_proportion_snvs = compute_shared_variant_proportion(tip1,tip2,wh_df_to_use)

            d = pd.DataFrame.from_dict({"tip1":[tip1],"tip2":[tip2],"tmrca":[tmrca_date],"clades_same":[clades_same],
                                        "divergence":[total_divergence],"prop_snvs_shared":[shared_proportion_snvs],
                                       "Ct_diff":[Ct_diff], "location1":[loc1],"location2":[loc2],
                                        "household_same":[same_household],"transmission_pair":[trans_pair],
                                       "great_circle_distance_km":[great_circle_distance],"location_same":[location],})

            df = df.append(d)
            combos.append(combo)

In [37]:
df.head()
len(df)

9591

In [40]:
# write to csv so I can use it in R 
df.to_csv("../data/WI-variants-vs-geo-2021-01-20.csv")

In [41]:
df.head()

,tip1,tip2,tmrca,clades_same,divergence,prop_snvs_shared,Ct_diff,location1,location2,household_same,transmission_pair,great_circle_distance_km,location_same
0,USA/WI-UW-22/2020,USA/WI-UW-306/2020,2019.976122,0,12.897647,0.0,NaN,,Milwaukee County,0,0,NaN,0
0,USA/WI-UW-22/2020,USA/WI-UW-118/2020,2019.976122,0,13.889686,0.0,NaN,,Sauk County,0,0,NaN,0
0,USA/WI-UW-22/2020,USA/WI-UW-282/2020,2019.976122,0,15.874464,0.0,NaN,,Milwaukee County,0,0,NaN,0
0,USA/WI-UW-22/2020,USA/WI-UW-337/2020,2019.976122,0,16.865709,0.0,NaN,,Milwaukee County,0,0,NaN,0
0,USA/WI-UW-22/2020,USA/WI-UW-798/2020,2019.976122,0,18.852395,0.0,NaN,,Dane County,0,0,NaN,0


## First, evaluate divergence 

I will first model shared diversity as a function of divergence. I will then model it as a combination of divergence, Ct differences and having the same clade

In [37]:
# evaluate the proportion of variants shared as a function of divergence
%R -i df
%R model.div = glm(prop_snvs_shared~divergence,data=df,family = gaussian(link="identity"),na.action(na.omit))
%R print(summary(model.div))

/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)



Call:
glm(formula = prop_snvs_shared ~ divergence, family = gaussian(link = "identity"), 
    data = df, weights = na.action(na.omit))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.22021  -0.10826  -0.04148   0.07466   0.77979  

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.2202096  0.0034286   64.23   <2e-16 ***
divergence  -0.0094025  0.0002532  -37.13   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.02079306)

    Null deviance: 230.94  on 9729  degrees of freedom
Residual deviance: 202.27  on 9728  degrees of freedom
AIC: -10069

Number of Fisher Scoring iterations: 2



In [61]:
# evaluate the proportion of variants shared as a function of great circle distance
%R -i df
%R model.geo = glm(prop_snvs_shared~great_circle_distance_km,data=df,family = gaussian(link="identity"),na.action(na.omit))
%R print(summary(model.geo))


Error in eval(predvars, data, env) : 
  object 'great_circle_distance_km' not found

Error in summary(model.geo) : object 'model.geo' not found


/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in eval(predvars, data, env) : 
  object 'great_circle_distance_km' not found

  warnings.warn(x, RRuntimeWarning)
/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in summary(model.geo) : object 'model.geo' not found

  warnings.warn(x, RRuntimeWarning)


In [100]:
# evaluate the proportion of variants shared as a function of Ct difference
%R -i df
%R model.div2 = glm(prop_snvs_shared~Ct_diff,data=df,family = gaussian(link="identity"),na.action(na.omit))
%R print(summary(model.div2))
%R print(anova(model.div2, test="Chisq"))


Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]) : 
  contrasts can be applied only to factors with 2 or more levels

Error in summary(model.div2) : object 'model.div2' not found

Error in anova(model.div2, test = "Chisq") : 
  object 'model.div2' not found


/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]) : 
  contrasts can be applied only to factors with 2 or more levels

  warnings.warn(x, RRuntimeWarning)


In [44]:
# lastly, try with clade same
%R -i df
%R model.div2 = glm(prop_snvs_shared~clades_same,data=df,family = gaussian(link="identity"),na.action(na.omit))
%R print(summary(model.div2))
%R print(anova(model.div2, test="Chisq"))


Call:
glm(formula = prop_snvs_shared ~ clades_same, family = gaussian(link = "identity"), 
    data = df, weights = na.action(na.omit))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.20805  -0.17344  -0.01959   0.12656   0.62656  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.208053   0.005015  41.489  < 2e-16 ***
clades_same -0.034615   0.005846  -5.921 3.44e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.02967363)

    Null deviance: 133.47  on 4464  degrees of freedom
Residual deviance: 132.43  on 4463  degrees of freedom
AIC: -3030.5

Number of Fisher Scoring iterations: 2



Analysis of Deviance Table

Model: gaussian, link: identity

Response: prop_snvs_shared

Terms added sequentially (first to last)


            Df Deviance Resid. Df Resid. Dev  Pr(>Chi)    
NULL                         4464     133.47              
clades_same  1   1.0402      4463     132.43 3.203e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## now try all together

In [102]:
# evaluate the proportion of variants shared as a linear combination of divergence and whether the clade is the same
%R -i df
%R model.marsh = glm(prop_snvs_shared~divergence+clades_same+Ct_diff+great_circle_distance_km,data=df,family = gaussian(link="identity"),na.action(na.omit))
%R print(summary(model.marsh))
%R print(coef(model.marsh))
%R print(anova(model.marsh, test="Chisq"))


Error in eval(predvars, data, env) : 
  object 'great_circle_distance_km' not found

Error in summary(model.marsh) : object 'model.marsh' not found

Error in coef(model.marsh) : object 'model.marsh' not found

Error in anova(model.marsh, test = "Chisq") : 
  object 'model.marsh' not found


/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in summary(model.marsh) : object 'model.marsh' not found

  warnings.warn(x, RRuntimeWarning)
/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in coef(model.marsh) : object 'model.marsh' not found

  warnings.warn(x, RRuntimeWarning)
/Users/lmoncla/anaconda/envs/LHM-basics/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in anova(model.marsh, test = "Chisq") : 
  object 'model.marsh' not found

  warnings.warn(x, RRuntimeWarning)


In [508]:
# evaluate the proportion of variants shared as a linear combination of divergence and whether the clade is the same
%R -i df
%R model.marsh = glm(prop_snvs_shared~divergence+clades_same+Ct_diff,data=df,family = gaussian(link="identity"),na.action(na.omit))
%R print(summary(model.marsh))
%R print(coef(model.marsh))
%R print(anova(model.marsh, test="Chisq"))


Call:
glm(formula = prop_snvs_shared ~ divergence + clades_same + Ct_diff, 
    family = gaussian(link = "identity"), data = df, weights = na.action(na.omit))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.19530  -0.08062  -0.02077   0.08819   0.28793  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.569131   0.023748  23.965   <2e-16 ***
divergence  -0.003896   0.002032  -1.917   0.0571 .  
clades_same  0.006918   0.022554   0.307   0.7595    
Ct_diff      0.003609   0.003121   1.156   0.2494    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.01235454)

    Null deviance: 1.9210  on 152  degrees of freedom
Residual deviance: 1.8408  on 149  degrees of freedom
AIC: -232.1

Number of Fisher Scoring iterations: 2



 (Intercept)   divergence  clades_same      Ct_diff 
 0.569130801 -0.003895600  0.006918280  0.003608546 


Analysis of Deviance Table

Model: gaussian, link: identity

Response: prop_snvs_shared

Terms added sequentially (first to last)


            Df Deviance Resid. Df Resid. Dev Pr(>Chi)  
NULL                          152     1.9210           
divergence   1 0.062623       151     1.8584  0.02436 *
clades_same  1 0.001026       150     1.8573  0.77322  
Ct_diff      1 0.016516       149     1.8408  0.24759  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [509]:
# evaluate the proportion of variants shared as a linear combination of divergence and whether the clade is the same
%R -i df
%R model.marsh = glm(prop_snvs_shared~great_circle_distance_km+clades_same+Ct_diff,data=df,family = gaussian(link="identity"),na.action(na.omit))
%R print(summary(model.marsh))
%R print(coef(model.marsh))
%R print(anova(model.marsh, test="Chisq"))


Call:
glm(formula = prop_snvs_shared ~ great_circle_distance_km + clades_same + 
    Ct_diff, family = gaussian(link = "identity"), data = df, 
    weights = na.action(na.omit))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.19447  -0.08212  -0.01615   0.07871   0.27799  

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.5659632  0.0231075  24.493   <2e-16 ***
great_circle_distance_km -0.0010747  0.0005681  -1.892   0.0605 .  
clades_same              -0.0044782  0.0200395  -0.223   0.8235    
Ct_diff                   0.0036859  0.0031183   1.182   0.2391    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.01236245)

    Null deviance: 1.921  on 152  degrees of freedom
Residual deviance: 1.842  on 149  degrees of freedom
AIC: -232

Number of Fisher Scoring iterations: 2



             (Intercept) great_circle_distance_km              clades_same 
             0.565963186             -0.001074677             -0.004478218 
                 Ct_diff 
             0.003685932 


Analysis of Deviance Table

Model: gaussian, link: identity

Response: prop_snvs_shared

Terms added sequentially (first to last)


                         Df Deviance Resid. Df Resid. Dev Pr(>Chi)  
NULL                                       152     1.9210           
great_circle_distance_km  1 0.060703       151     1.8603   0.0267 *
clades_same               1 0.001011       150     1.8593   0.7749  
Ct_diff                   1 0.017272       149     1.8420   0.2372  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
